In [27]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, mapping
from libpysal.weights import KNN
from spreg import ML_Lag
import seaborn as sns
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
import folium
import pyproj

In [28]:
property = gpd.read_file('data/cycle/Market_value_usable_hedonic_data.csv', engine='pyogrio')
canopy = gpd.read_file('data/canopy.gdb', engine='pyogrio')
ta = gpd.read_file('data/chc-boundaries/territorial-authority-2021-generalised.gpkg', engine='pyogrio')
sa2 = gpd.read_file('data/chc-boundaries/sa2/statistical-area-2-2023-generalised.shp')

In [ ]:
ta_chc = ta[ta['TA2021_V1_00_NAME_ASCII'] == 'Christchurch City']
sa2_chc = gpd.clip(sa2, ta_chc)

# suburbs = ['Inland water Lake Ellesmere/Te Waihora South', 'Southbridge', 'Inland water Lake Ellesmere/Te Waihora North', 'Motukarara', 'Tai Tapu', 'Kennedys Bush', 'Halswell South', 'Halswell South West', 'Halswell South East', 'Halswell North', 'Halswell West', 'Oaklands East', 'Oaklands West', 'Aidanfield', 'Ladbrooks', 'Trents', 'Newtons Road', 'Prebbleton North', 'Templeton', 'Broken Run', 'Awatea South', 'Wigram South', 'Wigram East', 'Hillmorton', 'Wigram West', 'Branston', 'Awatea North', 'Wigram North', 'Hornby South', 'Sockburn South', 'Middleton', 'Hornby Central', 'Upper Riccarton', 'Wharenui', 'Sockburn North', 'Riccarton Racecourse', 'Bush Inn', 'Ilam University', 'Ilam South', 'Islington-Hornby Industrial', 'Hornby West', 'Hei Hei', 'Islington', 'Paparua', 'Broomfield', 'Yaldhurst', 'Halkett', 'Mona Vale', 'Deans Bush', 'Holmwood', 'Merivale', 'St Albans West', 'Fendalton', 'Strowan', 'Rutland', 'Bryndwr South', 'Bryndwr North', 'Avonhead South', 'Russley', 'Avonhead East', 'Ilam North', 'Jellie Park', 'Avonhead West', 'Avonhead North', 'Burnside', 'Burnside Park', 'Burnside West', 'Malvern', 'Papanui East', 'Papanui West', 'Northlands (Christchurch City)', 'Northcote (Christchurch City)', 'Papanui North', 'Bishopdale North', 'Redwood East', 'Casebrook', 'Redwood West', 'Inlet Akaroa Harbour', 'Akaroa Harbour', 'Akaroa', 'Banks Peninsula Eastern Bays', 'Teddington', 'Diamond Harbour', 'Inlet Port Lyttelton', 'Lyttelton', 'Port Hills', 'Heathcote Valley', 'Banks Peninsula South', 'Governors Bay', 'Oceanic Canterbury Region', 'Cashmere East', 'Huntsbury', 'Beckenham', 'Hillsborough (Christchurch City)', 'St Martins', 'Opawa', 'Sydenham South', 'Westmorland', 'Cashmere West', 'Hoon Hay South', 'Somerfield West', 'Hoon Hay East', 'Hoon Hay West', 'Somerfield East', 'Spreydon South', 'Spreydon West', 'Spreydon North', 'Sumner', 'Clifton Hill', 'Inlets other Christchurch City', 'Mount Pleasant', 'Redcliffs', 'Woolston South', 'Brookhaven-Ferrymead', 'Woolston West', 'Woolston East', 'South New Brighton', 'Sydenham West', 'Sydenham North', 'Waltham', 'Ensors', 'Sydenham Central', 'Charleston (Christchurch City)', 'Lancaster Park', 'Phillipstown', 'Christchurch Central-South', 'Christchurch Central', 'Addington East', 'Addington West', 'Tower Junction', 'Addington North', 'Riccarton West', 'Riccarton South', 'Riccarton East', 'Riccarton Central', 'Hagley Park', 'Christchurch Central-West', 'Woolston North', 'Bromley South', 'Bromley North', 'Linwood East', 'Linwood North', 'Avonside', 'Bexley', 'Wainoni', 'New Brighton', 'Dallington', 'Stanmore', 'Linwood Central', 'Christchurch Central-East', 'Christchurch Central-North', 'Richmond South (Christchurch City)', 'St Albans East', 'Edgeware South', 'Richmond North (Christchurch City)', 'Edgeware North', 'St Albans North', 'Aranui', 'Otakaro-Avon River Corridor', 'Avondale (Christchurch City)', 'Rawhiti', 'Burwood South', 'North Beach', 'Burwood North', 'Waimairi Beach', 'Parklands', 'Queenspark', 'Mairehau South', 'Shirley West', 'Shirley East', 'Mairehau North', 'Prestons Park', 'Burlington-Oakbridge', 'Prestons', 'Marshland', 'Belfast East', 'Kaiapoi East', 'McLeans Island', 'Swannanoa-Eyreton', 'Eyrewell', 'Bishopdale South', 'Bishopdale West', 'Christchurch Airport', 'Harewood', 'Regents Park', 'Redwood North', 'Northwood', 'Clearwater', 'Belfast West', 'Clarkville', 'Waitikiri', 'Styx', 'Brooklands-Spencerville', 'Pegasus Bay']

# suburbs = ['Christchurch Central-South', 'Christchurch Central', 'Addington East', 'Addington West', 'Tower Junction', 'Addington North', 'Riccarton West', 'Riccarton South', 'Riccarton East', 'Riccarton Central', 'Hagley Park', 'Christchurch Central-West', 'Woolston North', 'Bromley South', 'Bromley North', 'Linwood East', 'Linwood North', 'Avonside', 'Bexley', 'Wainoni']

# suburbs = ['Avonside', 'Sydenham West', 'Sydenham North', 'Waltham', 'Ensors', 'Sydenham Central', 'Addington North', 'Riccarton West', 'Riccarton South', 'Riccarton East', 'Riccarton Central']

# suburbs = ['Avonside']

boundary = sa2_chc[sa2_chc['SA22023__2'].isin(suburbs)]

In [30]:
boundary.head()

,SA22023_V1,SA22023__1,SA22023__2,LAND_AREA_,AREA_SQ_KM,Shape_Leng,geometry
186,327100,Christchurch Central-South,Christchurch Central-South,1.250473,1.250473,6785.794476,"POLYGON ((1570157.215 5179155.893, 1570157.213..."
181,326600,Christchurch Central,Christchurch Central,0.713926,0.713926,4683.493839,"POLYGON ((1570882.391 5180158.623, 1570883.209..."
188,327400,Addington East,Addington East,0.568030,0.568030,3922.515449,"POLYGON ((1568958.779 5178876.807, 1569090.282..."
177,326100,Addington West,Addington West,0.553383,0.553383,4345.619737,"POLYGON ((1568450.049 5178817.224, 1568516.529..."
40,325500,Tower Junction,Tower Junction,1.077624,1.077624,5555.220648,"POLYGON ((1568142.413 5178866.956, 1568135.127..."


In [ ]:
property_2018 = property[property['YearSold'].isin(['2017', '2018', '2019'])]

property_gdf = gpd.GeoDataFrame(
  property_2018,
  geometry = gpd.points_from_xy(property_2018['gd2000co'], property_2018['gd2000_yco']),
  crs=4326
).to_crs(2193)

gdf_chc = gpd.clip(property_gdf, boundary)
canopy_chc = canopy.copy(deep=True)
# canopy_chc = gpd.clip(canopy, boundary)

In [32]:
# gdf_chc = gdf_chc.loc[:, ['GrossSalePrice','AgeAtSale', 'LandArea', 'TotalFloorArea', 'water_DIST', 'bus_DIST', 'Census_Pop', 'RnkIMDNoEm',
#        'RnkIMDNoIn', 'RnkIMDNoCr', 'RnkIMDNoHo', 'RnkIMDNoHe', 'RnkIMDNoEd',
#        'RnkIMDNoAc', 'CBD_DIST', 'geometry']]

gdf_chc = gdf_chc.loc[:, ['GrossSalePrice','AgeAtSale', 'LandArea', 'TotalFloorArea', 'water_DIST', 'bus_DIST', 'Census_Pop', 'RnkIMDNoEm',
       'RnkIMDNoIn', 'RnkIMDNoCr', 'RnkIMDNoHo', 'RnkIMDNoHe', 'RnkIMDNoEd',
       'RnkIMDNoAc', 'DECILE_high', 'DECILE_prim', 'Median_Income', 'CBD_DIST', 'geometry']]

In [34]:
gdf_chc["buf_20"]  = gdf_chc.geometry.buffer(20)
gdf_chc["buf_50"]  = gdf_chc.geometry.buffer(50)
gdf_chc["buf_100"] = gdf_chc.geometry.buffer(100)
gdf_chc["buf_200"] = gdf_chc.geometry.buffer(200)

def make_buffer_gdf(buf_series):
    return (gpd.GeoDataFrame({"geometry": buf_series}, crs=2193)
              .reset_index()
              .rename(columns={"index": "prop_id"}))

buf20  = make_buffer_gdf(gdf_chc["buf_20"])
buf50  = make_buffer_gdf(gdf_chc["buf_50"])
buf100 = make_buffer_gdf(gdf_chc["buf_100"])
buf200 = make_buffer_gdf(gdf_chc["buf_200"])


def canopy_area_overlay(canopy, buffers):
    inter = gpd.overlay(
        buffers,
        canopy,
        how="intersection",
        keep_geom_type=False
    )
    inter["area"] = inter.geometry.area
    return inter.groupby("prop_id")["area"].sum()

def canopy_area_sjoin(canopy, buffers):
    joined = gpd.sjoin(
        canopy,
        buffers,
        how="inner",
        predicate="intersects"
    )
    joined["area"] = joined.geometry.area
    return joined.groupby("prop_id")["area"].sum()

area_0_20  = canopy_area_overlay(canopy_chc, buf20)
area_0_50  = canopy_area_overlay(canopy_chc, buf50)
area_0_100 = canopy_area_sjoin(canopy_chc, buf100)
area_0_200 = canopy_area_sjoin(canopy_chc, buf200)

gdf_chc["canopy_0_20"]   = gdf_chc.index.map(area_0_20).fillna(0)
gdf_chc["canopy_0_50"]   = gdf_chc.index.map(area_0_50).fillna(0)
gdf_chc["canopy_0_100"]  = gdf_chc.index.map(area_0_100).fillna(0)
gdf_chc["canopy_0_200"]  = gdf_chc.index.map(area_0_200).fillna(0)

gdf_chc["canopy_20_50"]  = gdf_chc["canopy_0_50"] - gdf_chc["canopy_0_20"]
gdf_chc["canopy_50_100"]  = gdf_chc["canopy_0_100"] - gdf_chc["canopy_0_50"]
gdf_chc["canopy_100_200"] = gdf_chc["canopy_0_200"] - gdf_chc["canopy_0_100"]

In [ ]:
# gdf_chc['buffer_100m'] = gdf_chc.geometry.buffer(50)

# canopy_4326 = canopy_chc.to_crs(epsg='4326')
# gdf_4326 = gdf_chc.to_crs(epsg='4326')
# gdf_4326['buffer_100m'] = gdf_4326['buffer_100m'].to_crs(4326)

# minx, miny, maxx, maxy = canopy_4326.total_bounds

# lon = (minx + maxx) / 2
# lat = (miny + maxy) / 2
# m = folium.Map(location=(lat, lon), zoom_start=17, tiles=None)

# folium.TileLayer(
#   'cartodbpositron',
#   name='CartoDB Positron'
# ).add_to(m)

# for geom in gdf_4326['buffer_100m']:
#   folium.GeoJson(mapping(geom),
#                  style_function=lambda x: {
#                    'color': 'blue',
#                    'weight': 1,
#                    'fillOpacity': 0.1
#                  }).add_to(m)
  
# for geom in canopy_4326['geometry']:
#   folium.GeoJson(
#     mapping(geom),
#     style_function=lambda x: {
#       'color': 'green',
#       'weight': 1,
#       'fillOpacity': 0.3
#     }
#   ).add_to(m)

# for geom in gdf_4326.geometry:
#   folium.CircleMarker(
#     location=[geom.y, geom.x],
#     radius=3,
#     color='red'
#   ).add_to(m)

# m

In [35]:
gdf_chc.drop(inplace=True, columns=[
  'buffer_100m',
  'buf_20',
  'buf_50',
  'buf_100',
  'buf_200',
  'canopy_0_50',
  'canopy_0_100',
  'canopy_0_200',
  # 'canopy_0_20',
  # 'canopy_20_50',
  # 'canopy_50_100',
  # 'canopy_100_200',
], errors='ignore')

In [36]:
for col in gdf_chc.columns:
  if col != 'geometry':
    gdf_chc[col] = pd.to_numeric(gdf_chc[col], errors='coerce')

In [37]:
gdf_chc_scaled = gdf_chc[gdf_chc['LandArea'] != 0]

gdf_chc_scaled['log_price'] = np.log(gdf_chc['GrossSalePrice'])

numeric_cols = [
    col for col in gdf_chc.columns 
    if col not in ["geometry", "GrossSalePrice", "log_price"]
]

for col in numeric_cols:
    gdf_chc_scaled[col] = (gdf_chc[col] - gdf_chc[col].mean()) / gdf_chc[col].std()

/Users/rujalshrestha/Projects/chc-property-prices/venv/lib/python3.13/site-packages/geopandas/geodataframe.py:1968: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [38]:
X = gdf_chc_scaled[numeric_cols]

X = sm.add_constant(X)

y = gdf_chc_scaled["log_price"]

model = sm.OLS(y, X).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              log_price   R-squared:                       0.640
Model:                            OLS   Adj. R-squared:                  0.627
Method:                 Least Squares   F-statistic:                     49.60
Date:                Thu, 27 Nov 2025   Prob (F-statistic):          3.63e-115
Time:                        10:52:02   Log-Likelihood:                 204.79
No. Observations:                 609   AIC:                            -365.6
Df Residuals:                     587   BIC:                            -268.5
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             12.9576      0.015    852.411      0.000      12.928      12.987
AgeAtSale         -0.0757      0.008     -9.059      0.000      -0.092      -0.059
LandArea           0.0317      0.013      2.479      0.013       0.007       0.057
TotalFloorArea     0.0969      0.008     11.782      0.000       0.081       0.113
water_DIST         0.0038      0.009      0.398      0.691      -0.015       0.022
bus_DIST           0.0012      0.008      0.145      0.885      -0.015       0.017
Census_Pop         0.0415      0.008      5.467      0.000       0.027       0.056
RnkIMDNoEm        -0.0984      0.123     -0.802      0.423      -0.339       0.143
RnkIMDNoIn         0.1736      0.119      1.458      0.145      -0.060       0.407
RnkIMDNoCr        -0.2355      0.214     -1.101      0.271      -0.656       0.185
RnkIMDNoHo        -0.0378      0.131     -0.288      0.774      -0.295       0.220
RnkIMDNoHe         0.1038      0.117      0.891      0.373      -0.125       0.333
RnkIMDNoEd         0.3781      0.151      2.500      0.013       0.081       0.675
RnkIMDNoAc        -0.4451      0.673     -0.661      0.509      -1.767       0.877
DECILE_high       -0.0230      0.019     -1.219      0.223      -0.060       0.014
DECILE_prim        0.0471      0.016      3.023      0.003       0.016       0.078
Median_Income      0.0004      0.017      0.024      0.981      -0.032       0.033
CBD_DIST          -0.0677      0.018     -3.731      0.000      -0.103      -0.032
canopy_0_20       -0.0007      0.008     -0.082      0.935      -0.017       0.016
canopy_20_50       0.0343      0.011      3.133      0.002       0.013       0.056
canopy_50_100     -0.0111      0.010     -1.071      0.285      -0.031       0.009
canopy_100_200    -0.0075      0.013     -0.572      0.568      -0.033       0.018
==============================================================================
Omnibus:                      256.111   Durbin-Watson:                   1.405
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4173.956
Skew:                           1.421   Prob(JB):                         0.00
Kurtosis:                      15.507   Cond. No.                         306.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [39]:
print(gdf_chc['canopy_20_50'].std())
print(gdf_chc['GrossSalePrice'].mean())

421.1939723966729
414681.64129483816
